In [54]:
from ipynb.fs.full.data_extraction import read_file,init_spark
from pyspark.sql.functions import count,when,corr
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from matplotlib import pyplot as plt

In [29]:
spark = init_spark()

In [30]:
player = read_file("Player.csv")
player_attributes = read_file("Player_Attributes.csv")

In [31]:
player_grouped = player_attributes.groupBy("player_api_id").mean()
original_columns = player_attributes.columns

In [32]:
final_players = player_grouped.join(player,on="player_api_id")\
                .drop("player_fifa_api_id","id","birthday","avg(id)","avg(player_fifa_api_id)"
                      ,"avg(player_api_id)")

In [33]:
for col in original_columns:
    if col != "player_api_id":
        final_players = final_players.withColumnRenamed("avg("+col+")",col)

In [34]:
final_players = final_players.na.drop()

In [48]:
features = [x for x in final_players.columns if x not in ["player_name","player_api_id"]]
assembler = VectorAssembler(inputCols = features,outputCol="features")
final_data = assembler.transform(final_players)

In [49]:
kmeans = KMeans(distanceMeasure="cosine").setK(4).setSeed(1)
model = kmeans.fit(final_data.select("features"))
predictions = model.transform(final_data)
# final_players.select([count(when(final_players[c].isNull(), c)).alias(c) for c in final_players.columns]).show()

In [50]:
correlation = {}
for col in predictions.columns:
    if col!="features" and col!="prediction":
        c = predictions.select(corr(predictions[col],predictions["prediction"])).collect()[0][0]
        correlation[col] = c

In [67]:
# predictions.filter(predictions["prediction"]==1).select("player_name","overall_rating")\
# .orderBy("overall_rating",ascending=False).show()
train_pd = final_data.toPandas()
pred_pd = predictions.drop("features").toPandas()

In [68]:
pred_pd["player_name"] = pred_pd["player_name"].astype("str")

TypeError: '(slice(None, None, None), 0)' is an invalid key